In [4]:
import numpy as np
import pandas as pd

>We consider a list of 11 employees with employee IDs from 0 to 10.The 3 parameters in each embedded list is the staff_id,start time,duration.By a rule,every staff works for 10 hours each of the 5 working days of a week, hence the 5 rows.

In [5]:
staff_planning = [
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]]
]

>The staff needed is in the following shape:
+ list of days
+ each day is a list of 24 hours with the number of employees needed every hour

In [6]:
hourlystaff_needed = np.array([
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2]
])

### Check if an Employee is present


In [7]:
def employee_present(employee, time):
    employee_start_time = employee[1]
    employee_duration = employee[2]
    employee_end_time = employee_start_time + employee_duration
    if (time >= employee_start_time) and (time < employee_end_time):
        return True
    return False

### Convert a staff planning to a staff-needed planning.The employee planning is organised per employee, the staff-needed planning is the number of employees working per hour.The staff-needed planning is based on the employee planning and will allow to calculate the difference with the staff-needed


In [8]:
def staffplanning_to_hourlyplanning(staff_planning):
    
    hourlystaff_week = []
    for day in staff_planning:
        
        hourlystaff_day = [] 
        for employee in day:
            
            employee_present_hour = []
            for time in range(0, 24):
                
                employee_present_hour.append(employee_present(employee, time))
                
            hourlystaff_day.append(employee_present_hour)
            
        hourlystaff_week.append(hourlystaff_day)
        
    hourlystaff_week = np.array(hourlystaff_week).sum(axis = 1)  
    return hourlystaff_week

### The cost is calculated as hours understaffed + hours overstaffed

In [9]:
def cost(hourlystaff, hourlystaff_needed):
    errors = hourlystaff - hourlystaff_needed
    overstaff = abs(errors[errors > 0].sum())
    understaff = abs(errors[errors < 0].sum())
    
    overstaff_cost = 1
    understaff_cost = 1
    
    cost = overstaff_cost * overstaff + understaff_cost * understaff
    return cost

# CODE FOR GENETIC ALGORITHM


>## The random initialization
Given the fixed number of days and staffs,
The only randomised parameters are:
+ The start time of an Employee (between 0 and 23)
+ The duration of Work of each Employee (between 0 and 10)


In [10]:
def generate_random_staff_planning(n_days, n_staff):
    period_planning = []
    for day in range(n_days):
        day_planning = []
        for employee_id in range(n_staff):
            start_time = np.random.randint(0, 23)
            duration = np.random.randint(0, 10)
            employee = [employee_id, start_time, duration]
            day_planning.append(employee)

        period_planning.append(day_planning)
            
    return period_planning

# EXPOUND GENETICS

## STEP 1
### Genetic Creation
>Creation of n parent sapling

In [11]:
def create_parent_generation(n_parents, n_days = 7, n_staff = 11):
    parents = []
    for i in range(n_parents):
        parent = generate_random_staff_planning(n_days = n_days, n_staff = n_staff)
        parents.append(parent)
    return parents

## STEP 2
### Cross Over/Combination
>In each iteration, two parents are selected randomly and a random combination of the two parents is performed utilising th randomly generated yes/no mask on the parents that have been finalised.

In [12]:
def random_combine(parents, n_offspring):
    n_parents = len(parents)
    n_periods = len(parents[0])
    n_employees = len(parents[0][0])
    
    offspring = []
    for i in range(n_offspring):
        random_dad = parents[np.random.randint(low = 0, high = n_parents - 1)]
        random_mom = parents[np.random.randint(low = 0, high = n_parents - 1)]
        
        dad_mask = np.random.randint(0, 2, size = np.array(random_dad).shape)
        mom_mask = np.logical_not(dad_mask)
        
        child = np.add(np.multiply(random_dad, dad_mask), np.multiply(random_mom, mom_mask))

        offspring.append(child)
    return offspring

## STEP 3
### Mutation


In [13]:
def mutate_parent(parent, n_mutations):
    size1 = parent.shape[0]
    size2 = parent.shape[1]
    
    for i in range(n_mutations):

        rand1 = np.random.randint(0, size1)
        rand2 = np.random.randint(0, size2)
        rand3 = np.random.randint(1, 2)

        parent[rand1,rand2,rand3] = np.random.randint(0, 10)

    return parent

def mutate_gen(parent_gen, n_mutations):
    mutated_parent_gen = []
    for parent in parent_gen:
        mutated_parent_gen.append(mutate_parent(parent, n_mutations))
    return mutated_parent_gen

## STEP 4
### Feasibility
>Work duartion of more than 10 hours is not permitted

In [15]:
def is_acceptable(parent):
    return np.logical_not((np.array(parent)[:,:,2:] > 10).any()) 

def select_acceptable(parent_gen):
    parent_gen = [parent for parent in parent_gen if is_acceptable(parent)]
    return parent_gen

## STEP 5
### Selection Cost
> Using Inverse Fitness

In [16]:
def select_best(parent_gen, hourlystaff_needed, n_best):
    costs = []
    for idx, parent_staff_planning in enumerate(parent_gen):
        
        parent_hourly_planning = staffplanning_to_hourlyplanning(parent_staff_planning)
        parent_cost = cost(parent_hourly_planning, hourlystaff_needed)
        costs.append([idx, parent_cost])
    
    print('generations best is: {}, generations worst is: {}'.format(pd.DataFrame(costs)[1].min(), pd.DataFrame(costs)[1].max()))
    
    costs_tmp = pd.DataFrame(costs).sort_values(by = 1, ascending = True).reset_index(drop=True)
    selected_parents_idx = list(costs_tmp.iloc[:n_best,0])
    selected_parents = [parent for idx, parent in enumerate(parent_gen) if idx in selected_parents_idx]
    
    return selected_parents


# THE COMPLETE ALGORITHM

In [17]:
def gen_algo(hourlystaff_needed, n_iterations):   
    
    generation_size = 500
    
    parent_gen = create_parent_generation(n_parents = generation_size, n_days = 5, n_staff = 11)
    for it in range(n_iterations):
        parent_gen = select_acceptable(parent_gen)
        parent_gen = select_best(parent_gen, hourlystaff_needed, n_best = 100)
        parent_gen = random_combine(parent_gen, n_offspring = generation_size)
        parent_gen = mutate_gen(parent_gen, n_mutations = 1)
    
    best_child = select_best(parent_gen, hourlystaff_needed, n_best = 1)
    return best_child

## Executing...

In [18]:
best_planning = gen_algo(hourlystaff_needed, n_iterations = 100)

generations best is: 169, generations worst is: 260
generations best is: 163, generations worst is: 262
generations best is: 156, generations worst is: 233
generations best is: 152, generations worst is: 223
generations best is: 147, generations worst is: 217
generations best is: 142, generations worst is: 217
generations best is: 134, generations worst is: 210
generations best is: 131, generations worst is: 209
generations best is: 128, generations worst is: 195
generations best is: 121, generations worst is: 193
generations best is: 117, generations worst is: 183
generations best is: 116, generations worst is: 188
generations best is: 121, generations worst is: 185
generations best is: 118, generations worst is: 181
generations best is: 114, generations worst is: 181
generations best is: 106, generations worst is: 179
generations best is: 98, generations worst is: 171
generations best is: 101, generations worst is: 178
generations best is: 106, generations worst is: 164
generations b

In [19]:
print(best_planning)

[array([[[ 0,  4,  0],
        [ 1,  6,  3],
        [ 2, 17,  8],
        [ 3, 12,  8],
        [ 4,  6,  8],
        [ 5, 12,  8],
        [ 6, 18,  2],
        [ 7, 17,  9],
        [ 8, 11,  9],
        [ 9,  6,  7],
        [10,  6,  3]],

       [[ 0,  6,  3],
        [ 1, 17,  7],
        [ 2, 12,  8],
        [ 3,  8,  0],
        [ 4, 12,  8],
        [ 5, 12,  8],
        [ 6,  6,  8],
        [ 7,  6,  2],
        [ 8,  6,  9],
        [ 9,  7,  0],
        [10, 17,  9]],

       [[ 0, 17,  3],
        [ 1, 12,  8],
        [ 2, 17,  8],
        [ 3, 12,  8],
        [ 4,  6,  8],
        [ 5,  6,  3],
        [ 6, 17,  8],
        [ 7, 12,  2],
        [ 8, 17,  3],
        [ 9,  6,  2],
        [10,  6,  8]],

       [[ 0, 17,  3],
        [ 1,  6,  8],
        [ 2,  6,  8],
        [ 3,  7,  7],
        [ 4, 12,  2],
        [ 5, 12,  8],
        [ 6, 18,  8],
        [ 7, 12,  8],
        [ 8, 17,  7],
        [ 9, 19,  1],
        [10,  6,  2]],

       [[ 0,  7,  0],
 

In [20]:
print(staffplanning_to_hourlyplanning(best_planning[0]))

[[0 0 0 0 0 0 4 4 4 2 2 3 5 4 3 3 3 5 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 3 2 2 2 5 5 4 3 3 5 5 5 2 2 2 2]
 [0 0 0 0 0 0 4 4 3 2 2 2 5 5 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 3 4 3 3 3 3 6 6 2 2 2 4 5 6 2 2 2 2]
 [0 0 0 0 0 1 3 4 4 2 2 2 4 4 2 2 4 5 6 6 2 2 2 2]]


In [21]:
print(hourlystaff_needed)

[[0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]]
